In [9]:
import argparse
import csv
import json
import sys
from pathlib import Path
import tensorflow as tf
import numpy as np

cat_file = r"D:\David\Škola\RP\dataset\polyvore_outfits\categories.csv"

with open(cat_file) as categories:
    csv_reader = csv.reader(categories, delimiter=',')
    line_count = 0
    cat_groups = []
    cat_dict = {}
    for row in csv_reader:
            cat_number, cat, cat_group = row
            if cat_group.strip() not in cat_groups:
                cat_groups.append(cat_group.strip())
                
            cat_dict[int(cat_number)] = cat_group.strip()
                
            line_count += 1
        
    print(f'Total {len(cat_groups)} groups.')
    print(f'Total {line_count} categories.')

Total 12 groups.
Total 224 categories.


In [ ]:
dataset_path = r"D:\David\Škola\RP\dataset\polyvore_outfits\disjoint\test.json"
fitb_filepath = r"D:\David\Škola\RP\dataset\polyvore_outfits\disjoint\fill_in_blank_test.json"
dataset_root = r"D:\David\Škola\RP\dataset\polyvore_outfits"

def key_from_fitb_string(string):
    values = string.split("_")
    return int(values[0]), int(values[1])

with open(Path(dataset_root, "polyvore_item_metadata.json")) as json_file:
    metadata = json.load(json_file)

with open(dataset_path) as json_file:
    raw_json = json.load(json_file)
    print("Loaded " + str(len(raw_json)) + " items", flush=True)
    items = {}

    # Load all test items into dict
    for outfit in raw_json:
        set_id = int(outfit["set_id"])

        for item in outfit["items"]:
            items.update({(set_id, item["index"]): cat_dict[int(metadata[item["item_id"]]["category_id"])]})

examples = []
with open(fitb_filepath) as fitb_file:
    raw_json = json.load(fitb_file)
    print("Loaded " + str(len(raw_json)) + " questions", flush=True)

    # Compose questions from FITB file and test items dict
    for task in raw_json:
        set_id = None
        inputs = []
        input_categories = []
        targets = []
        target_categories = []
        target_pos = None

        for question_item_str in task["question"]:
            q_key = key_from_fitb_string(question_item_str)
            item_category = items[q_key]
            input_categories.append(item_category)
            set_id = q_key[0]
        pos = 0

        for question_item_str in task["answers"]:
            q_key = key_from_fitb_string(question_item_str)
            if q_key[0] == set_id:
                target_pos = pos
            item_category = items[q_key]
            target_categories.append(item_category)
            pos += 1
        
        if len(set(target_categories)) > 1:
            print(set_id)
            print(target_categories)
        